# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.50,79,95,5.84,SH,1702161478
1,1,socorro,31.6546,-106.3033,11.92,12,0,9.26,US,1702161478
2,2,mugia,43.1041,-9.2179,16.62,97,100,6.84,ES,1702161478
3,3,mirto,38.0847,14.7515,9.61,83,31,0.87,IT,1702161478
4,4,dikhil,11.5000,42.1667,20.63,69,0,5.95,DJ,1702161478


In [3]:
#Convert date into a more readable format
city_data_df['Date']=(pd.to_datetime(city_data_df['Date'],unit='s')).dt.date

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
plot_map= city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    titles= 'City',
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8
)

# Display the map
plot_map

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp']<=27)\
                            &(city_data_df['Max Temp']>=21)\
                            &(city_data_df['Cloudiness']<=1)\
                            &(city_data_df['Wind Speed']<4.5),:]
print(len(ideal_weather_df))

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

10


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,wailua homesteads,22.0669,-159.3780,26.24,75,0,2.57,US,2023-12-09
9,9,saint-pierre,-21.3393,55.4781,24.82,78,0,2.57,RE,2023-12-09
72,72,kapa'a,22.0752,-159.3190,26.99,75,0,2.57,US,2023-12-09
91,91,tsiombe,-25.3000,45.4833,21.68,84,0,1.71,MG,2023-12-09
216,216,guerrero negro,27.9769,-114.0611,24.52,32,0,0.49,MX,2023-12-09


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,6,wailua homesteads,22.0669,-159.3780,26.24,75,0,2.57,US,2023-12-09,
9,9,saint-pierre,-21.3393,55.4781,24.82,78,0,2.57,RE,2023-12-09,
72,72,kapa'a,22.0752,-159.3190,26.99,75,0,2.57,US,2023-12-09,
91,91,tsiombe,-25.3000,45.4833,21.68,84,0,1.71,MG,2023-12-09,
216,216,guerrero negro,27.9769,-114.0611,24.52,32,0,0.49,MX,2023-12-09,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accomodation.hotel',\
          'filter':'',\
          'bias':'',\
          'apiKey': geoapify_key,\
          'limit':2}  

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon= row['Lng']
    lat= row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
         hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
gewane - nearest hotel: No hotel found
les avirons - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,6,wailua homesteads,22.0669,-159.3780,26.24,75,0,2.57,US,2023-12-09,No hotel found
9,9,saint-pierre,-21.3393,55.4781,24.82,78,0,2.57,RE,2023-12-09,No hotel found
72,72,kapa'a,22.0752,-159.3190,26.99,75,0,2.57,US,2023-12-09,No hotel found
91,91,tsiombe,-25.3000,45.4833,21.68,84,0,1.71,MG,2023-12-09,No hotel found
216,216,guerrero negro,27.9769,-114.0611,24.52,32,0,0.49,MX,2023-12-09,No hotel found
259,259,salalah,17.0151,54.0924,24.05,43,0,3.60,OM,2023-12-09,No hotel found
534,534,gewane,10.1664,40.6453,22.67,53,0,2.14,ET,2023-12-09,No hotel found
539,539,les avirons,-21.2418,55.3394,23.05,78,0,2.57,RE,2023-12-09,No hotel found
564,564,dwarka,22.2394,68.9678,22.07,43,0,3.33,IN,2023-12-09,No hotel found
579,579,lihue,21.9789,-159.3672,26.58,69,0,2.57,US,2023-12-09,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
final_plot = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
final_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,City_ID,Max Temp,Cloudiness,Wind Speed,Country,Date,Hotel Name)